In [1]:
from delierium.Infinitesimals import prolongationODE
from delierium.JanetBasis import Janet_Basis
from delierium.helpers import is_derivative, order_of_derivative
import functools
from operator import mul

In [10]:
x = var('x')
y = function("y")
from IPython.core.debugger import set_trace
function('xi phi')
def infinitesimalsODE (ode, dependent, independent):
    prolongation = prolongationODE(ode, dependent, independent)[0].expand()
    # XXX: get correct order of ode
    s1                 = solve(ode==0, diff(dependent(independent),independent,independent, independent))
    determining_system = prolongation.subs({s1[0].lhs() : s1[0].rhs()}).simplify()
    determining_system.show()
    # so far it is checked manually and with mathematica
    #set_trace()
    all_derivative_monomials = set()
    for term in determining_system.operands():
        collector = 1
        for o in term.operands():
            if hasattr(o.operator(), "__name__") and (o.operator().__name__ == 'pow'):
                base, exponent = o.operands()
                if is_derivative(base) and base.operator().function() == dependent:
                    collector *= o
            elif is_derivative(o) and o.operator().function() == dependent:
                collector *= o
            else:
                pass
        if collector != 1:
            all_derivative_monomials.add(collector)
    class mon:
        def __init__ (self, it):
            self._it = it
        def __lt__ (self, other):
#            print("M"*77)
#            print(self._it, self._it.__class__, self._it.operator(), self._it.operands())
#            print(other._it, other._it.__class__,  other._it.operator(), other._it.operands())
            if is_derivative(self._it) and is_derivative(other._it):
                return order_of_derivative(self._it)[0] < order_of_derivative(other._it)[0]
            elif is_derivative(self._it) and other._it.operator().__name__ in ['mul_vararg', 'pow']:
                return True
            elif is_derivative(other._it) and self._it.operator().__name__ in ['mul_vararg', 'pow']:
                return False
            elif self._it.operator().__name__ == 'pow' == other._it.operator().__name__ :
                if self._it.operands()[0] == other._it.operands()[0]:
                    return self._it.operands()[1] < other._it.operands()[1]
                else:                
                    return self._it.operands()[1] < other._it.operands()[1]
            elif self._it.operator().__name__ == 'pow' and other._it.operator().__name__ == 'mul_vararg':
                return True
            elif self._it.operator().__name__ == 'mul_vararg' and other._it.operator().__name__ == 'pow':
                return False
            elif self._it.operator().__name__ == 'mul_vararg' and other._it.operator().__name__ == 'mul_vararg':
                if len(self._it.operands()) < len(other._it.operands()):
                    return False
                elif len(self._it.operands()) > len(other._it.operands()):
                    return True
                else:
                    s = self._it.operands()
                    o = other._it.operands()
                    s.sort()
                    o.sort()
                    for _s, _o in zip(s, o):
                        #set_trace()
                        if is_derivative(_s) and is_derivative(_o):
                            if order_of_derivative(_s)[0] < order_of_derivative(_o)[0]:
                                return True
                            else:
                                return False
                        elif is_derivative(_s) and not is_derivative(_o):
                            return True
                        elif is_derivative(_o) and not is_derivative(_s):
                            return False
                        elif _s.operator().__name__ == 'pow' == _o.operator().__name__:
                            left  = _s.operands()
                            right = _o.operands()
                            set_trace()
                            pass
                        else:
                            set_trace()
                            pass
            else:
                print("Shithole")
                return True
    vv = [_ for _ in reversed(sorted([mon(_) for _ in all_derivative_monomials]))]
    for _ in vv:
        print(_._it)
    #vv.append(vv[0])
    #del vv[0]
    hansi = []
    for v in vv:
        #set_trace()
        v = v._it
        coll = determining_system.expand().collect(v)
        schmauchi = 0
        for _coll in coll.operands():
            #_coll.show()
            #print(_coll.operands())
            if _coll == v:
                schmauchi = 1
                break
            else:
                husti = set(_coll.operands())
                #set_trace()
                if is_derivative (v) or v.operator().__name__ == 'pow':                    
                    _v = set([v])
                else:
                    _v = set(v.operands())
                if _v.issubset(husti):
                    print("HUHUUUUUU", v, husti)
                    schmauchi += functools.reduce(mul, husti - _v, 1)
                    break
        if schmauchi:
            hansi.append(schmauchi.expand().simplify())
        print("A"*88)
        schmauchi.show()
        v.show()
        (schmauchi * v).show()
        determining_system.expand().show()
        determining_system = determining_system - (schmauchi * v)
        determining_system.expand().show()
                
    #hansi.append(determining_system.expand())
    determining_system = hansi
    #determining_system = \
    #    [_ [0] for _ in determining_system.coefficients(diff(dependent(independent), independent))]
    # remove hardcoded xi, phi
    print ("this is my determining system")
    for _ in determining_system:
        _.simplify().show()
    #set_trace()
    janet = Janet_Basis(determining_system, [xi, phi], [dependent, independent])
    return janet
# Arrigo, Example 2.20
ode5=diff(y(x),x,x,x) + y(x)*diff(y(x),x,x)
inf = infinitesimalsODE(ode5, y ,x)
print("=========")
for _ in inf.S:
    _.show()
    _.Lder().show()

-y(x)*D[0, 0](xi)(y(x), x)*diff(y(x), x)^3 - D[0, 0, 0](xi)(y(x), x)*diff(y(x), x)^4 + y(x)*D[0, 0](phi)(y(x), x)*diff(y(x), x)^2 - 2*y(x)*D[0, 1](xi)(y(x), x)*diff(y(x), x)^2 + D[0, 0, 0](phi)(y(x), x)*diff(y(x), x)^3 - 3*D[0, 0, 1](xi)(y(x), x)*diff(y(x), x)^3 + y(x)*D[0](xi)(y(x), x)*diff(y(x), x)*diff(y(x), x, x) - 6*D[0, 0](xi)(y(x), x)*diff(y(x), x)^2*diff(y(x), x, x) + 2*y(x)*D[0, 1](phi)(y(x), x)*diff(y(x), x) - y(x)*D[1, 1](xi)(y(x), x)*diff(y(x), x) + 3*D[0, 0, 1](phi)(y(x), x)*diff(y(x), x)^2 - 3*D[0, 1, 1](xi)(y(x), x)*diff(y(x), x)^2 + y(x)*D[1](xi)(y(x), x)*diff(y(x), x, x) + 3*D[0, 0](phi)(y(x), x)*diff(y(x), x)*diff(y(x), x, x) - 9*D[0, 1](xi)(y(x), x)*diff(y(x), x)*diff(y(x), x, x) - 3*D[0](xi)(y(x), x)*diff(y(x), x, x)^2 + y(x)*D[1, 1](phi)(y(x), x) + 3*D[0, 1, 1](phi)(y(x), x)*diff(y(x), x) - D[1, 1, 1](xi)(y(x), x)*diff(y(x), x) + phi(y(x), x)*diff(y(x), x, x) + 3*D[0, 1](phi)(y(x), x)*diff(y(x), x, x) - 3*D[1, 1](xi)(y(x), x)*diff(y(x), x, x) + D[1, 1, 1](phi)(y(x), x)

diff(y(x), x)^2*diff(y(x), x, x)
diff(y(x), x)*diff(y(x), x, x)
diff(y(x), x)^4
diff(y(x), x)^3
diff(y(x), x, x)^2
diff(y(x), x)^2
diff(y(x), x, x)
diff(y(x), x)
HUHUUUUUU diff(y(x), x)^2*diff(y(x), x, x) {-6, diff(y(x), x)^2, D[0, 0](xi)(y(x), x), diff(y(x), x, x)}
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


-6*D[0, 0](xi)(y(x), x)

diff(y(x), x)^2*diff(y(x), x, x)

-6*D[0, 0](xi)(y(x), x)*diff(y(x), x)^2*diff(y(x), x, x)

...

-3*D[0](xi)(y(x), x)

y(x)*D[0, 0](phi)(y(x), x) - 2*y(x)*D[0, 1](xi)(y(x), x) + 3*D[0, 0, 1](phi)(y(x), x) - 3*D[0, 1, 1](xi)(y(x), x)

y(x)*D[1](xi)(y(x), x) + 3*D[0, 0](phi)(y(x), x)*diff(y(x), x) - 9*D[0, 1](xi)(y(x), x)*diff(y(x), x) + phi(y(x), x) + 3*D[0, 1](phi)(y(x), x) - 3*D[1, 1](xi)(y(x), x)

2*y(x)*D[0, 1](phi)(y(x), x) - y(x)*D[1, 1](xi)(y(x), x) + 3*D[0, 1, 1](phi)(y(x), x) - D[1, 1, 1](xi)(y(x), x)

TypeError: unsupported operand type(s) for *: 'sage.symbolic.expression.Expression' and 'NoneType'

In [ ]:
x = var('x')
y = function("y")
ode1=4*diff(y(x),x,x)*y(x) - 3* diff(y(x),x)**2-12*y(x)**3
inf = infinitesimalsODE(ode1, y ,x)

In [ ]:
for _ in inf.S:
    _.show()
    _.Lder().show()

In [ ]:
ode2=4*diff(y(x),x,x)*y(x)**2 - 3* (x**2)*diff(y(x),x)*2-12*y(x)**3
inf = infinitesimalsODE(ode2, y ,x)
print("=======")
for _ in inf.S:
    _.show()
    _.Lder().show()

In [ ]:
ode3=diff(y(x),x,x) + y(x)*diff(y(x),x)+x*y(x)**4
inf = infinitesimalsODE(ode3, y ,x)
print("=========")
for _ in inf.S:
    _.show()
    _.Lder().show()

In [ ]:
# Arrigo, Example 2.19
ode4=diff(y(x),x,x) + 3*y(x)*diff(y(x),x)+y(x)**3
inf = infinitesimalsODE(ode4, y ,x)
print("=========")
for _ in inf.S:
    _.show()
    #_.Lder().show()

In [ ]:
# Arrigo, Example 2.20
ode5=diff(y(x),x,x,x) + y(x)*diff(y(x),x,x)
inf = infinitesimalsODE(ode5, y ,x)
print("=========")
for _ in inf.S:
    _.show()
    _.Lder().show()